In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df = df.drop('customerID',axis=1)


In [ ]:
df.info()

> we have TotalCharge object we now is float so we transfer it float ignor nan values for now

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'].values, errors='coerce')

df.isnull().sum()

replace nan values mean

In [ ]:
df.loc[df['TotalCharges'].isna(),'TotalCharges'] = df['TotalCharges'].mean()

# Plot to understand data more******

In [ ]:
df1 = pd.get_dummies(df['Churn'], drop_first=True)
df2 = pd.concat([df.drop('Churn',axis=1),df1],axis=1)
df2.rename(columns = {'Yes' : 'Churn'}, inplace = True)

In [ ]:
plt.figure(figsize=(11,7))
plt.subplot(2,2,1)
sns.countplot(x='Contract',hue='Churn',data=df2)
plt.subplot(2,2,2)
sns.countplot(x='Partner',hue='Churn',data=df2)
plt.subplot(2,2,3)
sns.countplot(x='Dependents',hue='Churn',data=df2)
plt.subplot(2,2,4)
sns.countplot(x='PhoneService',hue='Churn',data=df2)

In [ ]:
plt.figure(figsize=(11,7))
plt.subplot(2,1,1)
sns.barplot(x='PaymentMethod',y='MonthlyCharges',hue='Churn',data=df2)
plt.subplot(2,1,2)
sns.barplot(x='PaymentMethod',y='TotalCharges',hue='Churn',data=df2)

In [ ]:
plt.subplot(2,1,1)
plot1 = df2.groupby(['Contract', 'Partner'])['Churn'].sum()
plot1.plot(kind='barh',figsize=(10, 9))
plt.subplot(2,1,2)
plot2 = df2.groupby(['Contract','PaymentMethod' ,'Partner'])['Churn'].sum()
plot2.plot(kind='barh',figsize=(10, 9))

In [ ]:
plt.subplot(2,1,1)
plot3 = df2.groupby(['Contract','Dependents' ,'Partner'])['Churn'].sum()
plot3.plot(kind='barh',figsize=(10, 9))
plt.subplot(2,1,2)
plot4 = df2.groupby(['Contract','PhoneService','Dependents' ,'Partner'])['Churn'].sum()
plot4.plot(kind='barh',figsize=(10, 9))

# Preprocessing Data
Split data to categorial and numeric for encoding categorical

In [ ]:
X = df.iloc[:,0:19]
Y = df2.iloc[:,-1].values

split te numeric data and categorial data for encoder

In [ ]:
Featur_cat_X = X.drop(['SeniorCitizen','tenure','MonthlyCharges','TotalCharges'],axis=1)
Featur_num_X = X.iloc[:,[4,17,18]].values

In [ ]:
sc = StandardScaler()
oneh = OneHotEncoder()
X_x = oneh.fit_transform(Featur_cat_X).todense()

drope first coulmns minimize features

In [ ]:
oneh = OneHotEncoder(drop='first')
X_x = oneh.fit_transform(Featur_cat_X).todense()

concate numeric and encoding categorial to get X

In [ ]:
x = np.concatenate((X_x,Featur_num_X), axis=1)

*scaling data***

In [ ]:
XX1 = sc.fit_transform(x)


In [ ]:
X_train, X_test, y_train,y_test= train_test_split(XX1,Y,test_size=0.2,random_state=0)

In [ ]:
def acuray(model):
    model.fit(X_train,y_train)
    print(classification_report(y_test,model.predict(X_test)))
    print('model {}'.format(model))
    print(model.score(X_test, y_test))

In [ ]:
# test acccuracy our model
acuray(LogisticRegression(penalty='l2',solver='lbfgs',l1_ratio=0))
acuray(AdaBoostClassifier())
acuray(SVC(kernel='linear'))


#we have imbalance data test in 1 only 0.57 accurcy

In [ ]:
smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm=smote.fit_resample(XX1,Y)

In [ ]:
X_train, X_test, y_train,y_test= train_test_split(X_sm, y_sm,test_size=0.2,random_state=15)


In [ ]:
acuray(LogisticRegression(penalty='l2',solver='lbfgs',l1_ratio=0))
acuray(AdaBoostClassifier())
acuray(RandomForestClassifier())
acuray(XGBClassifier())

after balanced data XGboost and randomforest best modul
before balanced data LogisticRegression best module